In [1]:
%load_ext lab_black
import time


def do_something(wait):
    print("begin something")
    time.sleep(wait)
    print("something finished")


print("\n\n-- normal function --\n\n")
start = time.perf_counter()
do_something(1)
do_something(1)
finish = time.perf_counter()
print(f"{round(finish - start, 2)} secods elapsed")



-- normal function --


begin something
something finished
begin something
something finished
2.0 secods elapsed


In [2]:
import multiprocessing


def do_something_with_return(wait):
    print("begin something {}".format(wait))
    time.sleep(wait)
    return f"something finished in {wait} seconds"


print("\n\n-- parallel function --\n\n")
start = time.perf_counter()

process_1 = multiprocessing.Process(target=do_something_with_return, args=[1])
process_2 = multiprocessing.Process(target=do_something_with_return, args=[1])
process_1.start()
process_2.start()
process_1.join()
process_2.join()

finish = time.perf_counter()
print(f"{round(finish - start, 2)} secods elapsed")



-- parallel function --


begin something 1
begin something 1
1.02 secods elapsed


In [3]:
import multiprocessing

print("\n\n-- parallel with list function --\n\n")
start = time.perf_counter()

processes = []

for _ in range(10):
    process = multiprocessing.Process(target=do_something_with_return, args=[1])
    process.start()
    processes.append(process)

for current_process in processes:
    current_process.join()


finish = time.perf_counter()
print(f"{round(finish - start, 2)} secods elapsed")



-- parallel with list function --


begin something 1begin something 1
begin something 1begin something 1begin something 1


begin something 1begin something 1begin something 1begin something 1begin something 1





1.04 secods elapsed


In [4]:
# Running Parallel using concurent packages will spend more time, because the packages can make decision wheter we should use all processor or not

from concurrent import futures

print("\n\n-- parallel with futures function --\n\n")
start = time.perf_counter()

with futures.ProcessPoolExecutor() as executor:
    seconds = [1, 2, 3, 4]
    results = [executor.submit(do_something_with_return, sec) for sec in seconds]
    for f in futures.as_completed(results):
        print(f.result())

finish = time.perf_counter()
print(f"{round(finish - start, 2)} secods elapsed")



-- parallel with futures function --


begin something 2begin something 1begin something 3begin something 4



something finished in 1 seconds
something finished in 2 seconds
something finished in 3 seconds
something finished in 4 seconds
4.05 secods elapsed


In [5]:
print("\n\n-- parallel with futures map function --\n\n")
start = time.perf_counter()

with futures.ProcessPoolExecutor() as executor:
    seconds = [1, 2, 3, 4]
    results = executor.map(do_something_with_return, seconds)
    for result in results:
        print(result)

finish = time.perf_counter()
print(f"{round(finish - start, 2)} secods elapsed")



-- parallel with futures map function --


begin something 3begin something 2begin something 1begin something 4



something finished in 1 seconds
something finished in 2 seconds
something finished in 3 seconds
something finished in 4 seconds
4.03 secods elapsed


In [6]:
import time
import concurrent.futures
from PIL import Image, ImageFilter
from os import listdir

path = "./download"
img_names = ["{}/{}".format(path, img) for img in listdir(path)]

print(img_names)

t1 = time.perf_counter()

size = (1200, 1200)


def process_image(img_name):
    img = Image.open(img_name)
    img = img.filter(ImageFilter.GaussianBlur(15))
    #     img.thumbnail(size)
    img.save(f"{img_name}-edited.jpg")
    print(f"{img_name} was processed...")


with concurrent.futures.ProcessPoolExecutor() as executor:
    executor.map(process_image, img_names)


t2 = time.perf_counter()

print(f"Finished in {t2-t1} seconds")

['./download/2020-03.jpg-edited.jpg', './download/2020-03.jpg', './download/data.jpg', './download/2020-03.jpg-edited.jpg-edited.jpg']
./download/2020-03.jpg-edited.jpg-edited.jpg was processed...
./download/2020-03.jpg-edited.jpg was processed..../download/2020-03.jpg was processed...

Finished in 0.05616811999971105 seconds


In [7]:
from concurrent import futures
import time
import threading
import multiprocessing


def compute(duration):
    time.sleep(duration)
    return "{} seconds elapsed in thread {} and in process {}".format(
        duration, threading.currentThread().name, multiprocessing.current_process().name
    )


def create_thread(duration):
    print("enter thread")
    with futures.ThreadPoolExecutor() as executor:
        arguments = [duration] * 10
        results = executor.map(compute, arguments)
        for result in results:
            print(result)


def main():
    with futures.ProcessPoolExecutor() as executor:
        arguments = [3] * 5
        executor.map(create_thread, arguments)


main()

enter threadenter threadenter threadenter thread



3 seconds elapsed in thread ThreadPoolExecutor-0_0 and in process ForkProcess-263 seconds elapsed in thread ThreadPoolExecutor-0_0 and in process ForkProcess-27
3 seconds elapsed in thread ThreadPoolExecutor-0_0 and in process ForkProcess-253 seconds elapsed in thread ThreadPoolExecutor-0_0 and in process ForkProcess-28

3 seconds elapsed in thread ThreadPoolExecutor-0_1 and in process ForkProcess-26

3 seconds elapsed in thread ThreadPoolExecutor-0_2 and in process ForkProcess-263 seconds elapsed in thread ThreadPoolExecutor-0_1 and in process ForkProcess-253 seconds elapsed in thread ThreadPoolExecutor-0_1 and in process ForkProcess-27
3 seconds elapsed in thread ThreadPoolExecutor-0_1 and in process ForkProcess-283 seconds elapsed in thread ThreadPoolExecutor-0_3 and in process ForkProcess-26

3 seconds elapsed in thread ThreadPoolExecutor-0_2 and in process ForkProcess-283 seconds elapsed in thread ThreadPoolExecutor-0_4 and in pr